In [297]:
import numpy as np
from PIL import Image
import scipy
import cv2

ImportError: libGL.so.1: cannot open shared object file: No such file or directory

In [43]:
def normalize(p):
    return p / np.sum(p)

In [44]:
def dist(x, y, alpha):
    return ((x[0]-y[0])**alpha + (x[1]-y[1])**alpha)

In [45]:
def Cost_matrix(N, alpha):
    C = np.zeros((N*N, N*N))
    for j1 in range(N):
        for j2 in range(N):
            column = []
            for i in range(N):
                column.append(dist([0, j1], [i, j2], alpha))
            C[N*j1:N*j1+N, N*j2:N*j2+N] = scipy.linalg.toeplitz(np.array(column))
    return(C)

In [46]:
def multiply(K, X): # Takes in argument a matrix of cost K and a vector X
    n = int(np.sqrt(len(X)))
    Y = np.zeros(len(K[0]))
    for j in range(n):
        for i in range(n):
            Y[n*j:n*j+n] += scipy.linalg.matmul_toeplitz(K[n*i:n*i+n, n*j:n*j+n][0], X[n*j:n*j+n])
    return Y

In [247]:
def sinkhorn_differentiate(Lambda, p, q, Gamma, L):

    S = len(p)
    N = len(p[0])
    C = Cost_matrix(int(np.sqrt(N)), 2)
    K = np.exp(- C / Gamma)
    Phi = np.ones((L+1, N, S))
    b = np.ones((L+1, N, S))
    w = np.zeros(S)
    r = np.zeros((N, S))
    Lambda0 = normalize(np.exp(np.copy(Lambda)))
    
    for i in range(1, L+1):
        P = np.ones(N)

        for j in range(S):
            Phi[i, :, j] = multiply(K, (p[j] / multiply(K, b[i-1, :, j])))
            P *= Phi[i, :, j]**Lambda0[j]
        
        for j in range(S):
            b[i, :, j] = P / Phi[i, :, j]

    g = 2*(P - q) * P
    
    for i in range(1, L+1):

        for j in range(S):
            w[j] += np.inner(np.log(Phi[L-i+1, :, j]), g)
            r[:, j] = - multiply(K, (multiply(K, ((Lambda0[j] * g - r[:, j]) / Phi[L-i+1, :, j])) * (p[j] / multiply(K, b[L-i, :, j]) ** 2))) * b[L-i, :, j]
       
        g = np.zeros(N)
        for j in range(S):    
            g += r[:, j]

    return np.sum((P - q) ** 2), w

In [258]:
img1 = Image.open('Images/ML_HOGGAR/HOGGAR_data1.jpg')
img2 = Image.open('Images/ML_HOGGAR/HOGGAR_data2.jpg')
img3 = Image.open('Images/ML_HOGGAR/HOGGAR_data3.jpg')
img4 = Image.open('Images/ML_HOGGAR/HOGGAR_data5.jpg')

shape1 = np.asarray(img1).reshape(-1)
shape2 = np.asarray(img2).reshape(-1)
shape3 = np.asarray(img3).reshape(-1)
shape4 = np.asarray(img4).reshape(-1)

hist1 = np.histogram(shape1, bins=np.arange(0,256), density=True)[0]
hist2 = np.histogram(shape2, bins=np.arange(0,256), density=True)[0]
hist3 = np.histogram(shape3, bins=np.arange(0,256), density=True)[0]
hist4 = np.histogram(shape4, bins=np.arange(0,256), density=True)[0]

hist1 = np.append(np.zeros(1), hist1)
hist2 = np.append(np.zeros(1), hist2)
hist3 = np.append(np.zeros(1), hist3)
hist4 = np.append(np.zeros(1), hist4)

hists = np.array([hist1, hist2, hist3])

In [263]:
shape = np.asarray(Image.open('Images/ML_HOGGAR/HOGGAR_obj.jpg')).reshape(-1)
input_image = np.histogram(shape, bins=np.arange(0,256), density=True)[0]
input_image = np.append(np.zeros(1), input_image)
q = input_image

In [239]:
Lambdas = np.array([0.5 for i in range(len(hists))])
sinkhorn_differentiate(Lambdas, C, hists, q, 20, 20)

(array([0.6708116 , 0.91629615, 1.10586609, 1.21014627, 1.2357614 ,
        1.21189204, 1.17154485, 1.13996834, 1.129508  , 1.13872608,
        1.15534068, 1.15938415, 1.12538176, 1.02954239, 0.86439727,
        0.65029076, 0.3191802 , 0.47220862, 0.63157373, 0.77698133,
        0.89543942, 0.98494731, 1.05280574, 1.11062467, 1.16853373,
        1.22984003, 1.28658416, 1.31753304, 1.29227501, 1.18448326,
        0.99044513, 0.73957537, 0.50679161, 0.70056904, 0.87174877,
        0.99781157, 1.07501432, 1.11475878, 1.13439637, 1.14928355,
        1.16829714, 1.19161189, 1.20947337, 1.20264827, 1.14716685,
        1.02520493, 0.83854135, 0.61454497, 0.5649495 , 0.77388072,
        0.94910982, 1.06451288, 1.11841117, 1.12856751, 1.11933037,
        1.11071547, 1.1132982 , 1.1271269 , 1.14187702, 1.13791741,
        1.09075994, 0.9812351 , 0.80863403, 0.59734625, 0.57779526,
        0.7934088 , 0.97388978, 1.09152939, 1.14432685, 1.15076265,
        1.13618075, 1.12131943, 1.11711604, 1.12

In [266]:
p = hists
LambdaINI = np.array([0.3, 0.4, 0.3])
resH = scipy.optimize.minimize(sinkhorn_differentiate, LambdaINI, method='BFGS', args=(p, q, 10, 10), jac=True)

In [267]:
print(resH)

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 0.002414900630069163
        x: [ 1.383e+00 -1.119e+00 -6.910e-01]
      nit: 19
      jac: [ 1.032e-04 -3.148e-04  1.027e-04]
 hess_inv: [[ 1.399e+03  2.494e+01  5.439e+03]
            [ 2.494e+01  1.346e+02  5.489e+02]
            [ 5.439e+03  5.489e+02  2.269e+04]]
     nfev: 80
     njev: 69


In [268]:
LambdaH = normalize(np.exp(resH.x))
print(LambdaH)

[0.82804178 0.06784739 0.10411083]


In [295]:
def sinkhorn(Lambda, hists, input_image, Gamma, L, epsilon):

    N = len(hists[0])
    C = Cost_matrix(int(np.sqrt(N)), 2)
    q, w = sinkhorn_differentiate(Lambda, hists, input_image, Gamma, L)
    K = np.exp(-C/Gamma)
    
    u = np.ones(len(C[0]))
    v = np.ones(len(C[1]))
    T = np.diag(u.flatten()) @ K @ np.diag(v.flatten())
    t_norm = np.trace(T.T @ T)
    
    while True:
        u = input_image/multiply(K, v) # avoid divided by zero
        v = q/multiply(K, u)
        T = np.diag(u.flatten()) @ K @ np.diag(v.flatten())
        if abs((np.trace(T.T @ T) - t_norm)/t_norm) < epsilon:
            break
        t_norm = np.trace(T.T @ T)
    return T, np.trace(C.T @ T)

In [299]:
sinkhorn(LambdaH, hists, input_image, 10, 10, 1)

(array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [6.36341830e-04, 5.78886013e-04, 4.95646244e-04, ...,
         1.08443812e-20, 7.04137841e-22, 4.47467756e-23],
        [6.71645542e-05, 7.46279680e-05, 7.80439562e-05, ...,
         1.54106194e-20, 1.22217078e-21, 9.48625633e-23],
        ...,
        [1.69143699e-21, 1.69615355e-20, 1.60085016e-19, ...,
         3.57280777e-05, 2.55722719e-05, 1.79134870e-05],
        [3.92077459e-22, 4.80219854e-21, 5.53585272e-20, ...,
         1.11504232e-04, 9.74787413e-05, 8.34026073e-05],
        [1.58525182e-22, 2.37151237e-21, 3.33909447e-20, ...,
         6.06992594e-04, 6.48128097e-04, 6.77313076e-04]], shape=(256, 256)),
 np.float64(7.397769835943807))

In [300]:
def transformation(x, T):
    

SyntaxError: incomplete input (4061780513.py, line 2)